In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from datetime import datetime as dt
import os

def getNow(folder='logFolder'):
    now       = dt.now().strftime('%Y-%m-%d--%H-%M-%S')
    nowFolder = os.path.join('./{}'.format(folder), now)
    return nowFolder

In [2]:
N = 1000
X = np.random.rand(N, 2)
y = 2*X[:, 0] + 3*X[:, 1] + 4 + 0.1*(np.random.rand(N)-0.5)
y = y.reshape((-1, 1))

In [6]:
tf.reset_default_graph()

In [7]:
# Generate a neural network
inpSize     = X.shape[1]
layers      = [10, 5, 1]
activations = [tf.tanh, tf.tanh, None]

with tf.variable_scope('io'):
    inp = tf.placeholder(dtype=tf.float32, shape=(None, 2), name='inp')
    out = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='out')

prevL = inpSize
with tf.variable_scope('model'):
    NN = inp * 1
    for i, (l, a) in enumerate(zip(layers, activations)):
        with tf.variable_scope('layer_{:05d}'.format(i)):
            
            W = tf.Variable(initial_value=np.random.rand(prevL, l), name='W', dtype=tf.float32)
            b = tf.Variable(initial_value=np.zeros((1, l)),         name='b', dtype=tf.float32)
            NN = tf.matmul(NN, W) + b
            if a is not None:
                NN = a(NN)

            prevL = l

    err  = tf.reduce_mean((out - NN)**2, name='err')
    
with tf.variable_scope('optimizers'):
    opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(err)

with tf.variable_scope('misc'):
    init  = tf.global_variables_initializer()
    saver = tf.train.Saver(var_list=tf.trainable_variables())
    trainableVars = tf.trainable_variables()

In [8]:
summaryFolder = getNow()
modelFolder   = getNow('models')

with tf.Session() as sess:
    writer = tf.summary.FileWriter(summaryFolder, sess.graph)
    sess.run(init)
    
    for i in range(1000):
        _, errVal = sess.run( [opt, err], feed_dict={
            inp : X, out : y})
        
        if i % 100 == 0:
            print(errVal)
            
    os.makedirs(modelFolder)
    path = saver.save( sess, os.path.join( modelFolder, 'model.ckpt' ) )
        
    writer.close()

print(errVal)    
print('tensorboard --logdir={}'.format( summaryFolder ))
print('savePath = {}'.format( path ))

9.826427
0.6140118
0.15328982
0.06865464
0.016933467
0.010874928
0.007163512
0.005136735
0.0041266982
0.0035379939
0.0031211253
tensorboard --logdir=./logFolder/2019-05-10--16-43-21
savePath = ./models/2019-05-10--16-43-21/model.ckpt


In [8]:
!ls ./models/2019-02-11--18-58-58

checkpoint                     model.ckpt.index
model.ckpt.data-00000-of-00001 model.ckpt.meta
